In [72]:
#!pip install tk tkcalendar

In [73]:
import sqlite3
import tkinter as tk
from tkcalendar import Calendar
import xml.etree.ElementTree as ET
from tkinter import ttk, messagebox
import datetime

In [74]:
# ---------------------------
# 1. Initialize Database
# ---------------------------
def init_db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE,
                        password TEXT)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS events (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        user_id INTEGER,
                        title TEXT,
                        start_time TEXT,
                        end_time TEXT,
                        is_shared INTEGER DEFAULT 0)''')
    conn.commit()
    conn.close()

def get_connection():
    return sqlite3.connect(DB_NAME)

init_db()


In [75]:
# ---------------------------
# 2. Authentication Functions
# ---------------------------
def register_user(username, password):
    conn = get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO users(username,password) VALUES(?,?)", (username, password))
        conn.commit()
        return True
    except:
        return False
    finally:
        conn.close()

def login(username, password):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))
    user = cursor.fetchone()
    conn.close()
    return user


In [76]:
# ---------------------------
# 3. Event Management
# ---------------------------
def add_event(user_id, title, start_time, end_time, is_shared=0):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO events(user_id,title,start_time,end_time,is_shared) VALUES(?,?,?,?,?)",
                   (user_id, title, start_time, end_time, is_shared))
    conn.commit()
    conn.close()

def get_events(user_id):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT id, title, start_time, end_time, is_shared FROM events WHERE user_id=?", (user_id,))
    events = cursor.fetchall()
    conn.close()
    return events


In [77]:
# ---------------------------
# 4. Export Shared Events to XML
# ---------------------------
def export_shared_events():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT title,start_time,end_time FROM events WHERE is_shared=1")
    events = cursor.fetchall()
    conn.close()

    root = ET.Element("Events")
    for ev in events:
        e = ET.SubElement(root, "Event")
        ET.SubElement(e, "Title").text = ev[0]
        ET.SubElement(e, "StartTime").text = ev[1]
        ET.SubElement(e, "EndTime").text = ev[2]

    tree = ET.ElementTree(root)
    tree.write("shared_calendar.xml")
    messagebox.showinfo("Export", "Shared events exported to shared_calendar.xml")


In [78]:
# ---------------------------
# 5. GUI with Better Layout
# ---------------------------
current_user = None

def show_calendar():
    global cal_window
    cal_window = tk.Toplevel()
    cal_window.title("Calendar Dashboard")
    cal_window.geometry("800x500")

    # Frame for Calendar
    left_frame = ttk.Frame(cal_window, padding=10)
    left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    # Calendar widget
    cal = Calendar(left_frame, selectmode='day')
    cal.pack(pady=20)

    # Frame for Event Form
    right_frame = ttk.Frame(cal_window, padding=10)
    right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

    ttk.Label(right_frame, text="Event Title:").pack(anchor=tk.W)
    title_entry = ttk.Entry(right_frame, width=30)
    title_entry.pack(pady=5)

    ttk.Label(right_frame, text="Start Time (YYYY-MM-DD HH:MM):").pack(anchor=tk.W)
    start_entry = ttk.Entry(right_frame, width=30)
    start_entry.pack(pady=5)

    ttk.Label(right_frame, text="End Time (YYYY-MM-DD HH:MM):").pack(anchor=tk.W)
    end_entry = ttk.Entry(right_frame, width=30)
    end_entry.pack(pady=5)

    share_var = tk.IntVar()
    ttk.Checkbutton(right_frame, text="Share Event", variable=share_var).pack(pady=5)

    def add_new_event():
        title = title_entry.get()
        start_time = start_entry.get()
        end_time = end_entry.get()
        if not title or not start_time or not end_time:
            messagebox.showerror("Error", "Please fill all fields")
            return
        add_event(current_user[0], title, start_time, end_time, share_var.get())
        messagebox.showinfo("Success", "Event added successfully")
        load_events()

    ttk.Button(right_frame, text="Add Event", command=add_new_event).pack(pady=10)
    ttk.Button(right_frame, text="Export Shared Events", command=export_shared_events).pack(pady=5)

    # Event List Table
    ttk.Label(cal_window, text="Your Events", font=("Arial", 12, "bold")).pack(side=tk.BOTTOM, anchor=tk.W)
    columns = ("ID", "Title", "Start", "End", "Shared")
    event_table = ttk.Treeview(cal_window, columns=columns, show='headings')
    for col in columns:
        event_table.heading(col, text=col)
    event_table.pack(fill=tk.X, side=tk.BOTTOM)

    def load_events():
        for row in event_table.get_children():
            event_table.delete(row)
        for ev in get_events(current_user[0]):
            event_table.insert("", tk.END, values=ev)

    load_events()


In [79]:
# ---------------------------
# 6. Login & Register UI
# ---------------------------
def login_screen():
    global current_user
    username = user_entry.get()
    password = pass_entry.get()
    user = login(username, password)
    if user:
        current_user = user
        messagebox.showinfo("Success", "Login successful!")
        show_calendar()
    else:
        messagebox.showerror("Error", "Invalid credentials")

def register_screen():
    username = user_entry.get()
    password = pass_entry.get()
    if register_user(username, password):
        messagebox.showinfo("Success", "Registered successfully!")
    else:
        messagebox.showerror("Error", "Username already exists")


In [80]:
# ---------------------------
# 7. Main Login Window
# ---------------------------
root = tk.Tk()
root.title("Synchronized Desktop Calendar")
root.geometry("300x200")

ttk.Label(root, text="Synchronized Calendar", font=("Arial", 14)).pack(pady=10)

user_entry = ttk.Entry(root, width=25)
user_entry.insert(0, "PKsaxena")
user_entry.pack(pady=5)

pass_entry = ttk.Entry(root, width=25, show="*")
pass_entry.insert(0, "Jack")
pass_entry.pack(pady=5)

ttk.Button(root, text="Login", command=login_screen).pack(pady=5)
ttk.Button(root, text="Register", command=register_screen).pack(pady=5)

root.mainloop()
